# Hazard map

In [19]:
#### version 2

import rasterio
import numpy as np

# Function to apply a buffer to a raster and write the buffer distance as the value
def buffer_raster(input_raster, output_raster, buffer_distance):
    # Open the raster file
    with rasterio.open(input_raster) as src:
        # Read the raster data
        data = src.read(1)  # Read the first band (usually the data you want)
        
        # Get raster metadata
        profile = src.profile
        
        # Get pixel size (assuming square cells)
        pixel_size = abs(profile['transform'][0])  # cell size in map units (e.g., meters)
        
        # Buffer distance in pixels
        buffer_pixels = int(buffer_distance / pixel_size)
        
        # Create a kernel (a square window) to apply around each pixel
        kernel_size = 2 * buffer_pixels + 1
        kernel = np.ones((kernel_size, kernel_size), dtype=np.uint8)
        
        # Create an output array for the buffered raster
        buffered_data = np.zeros_like(data, dtype=np.float32)
        
        # Apply the kernel as a convolution (simple box filter)
        for i in range(buffer_pixels, data.shape[0] - buffer_pixels):
            for j in range(buffer_pixels, data.shape[1] - buffer_pixels):
                # Define the region of interest (ROI) around the current pixel
                roi = data[i-buffer_pixels:i+buffer_pixels+1, j-buffer_pixels:j+buffer_pixels+1]
                
                # If any of the ROI values are non-zero (indicating presence), set the pixel as the buffer distance
                if np.any(roi > 0):  # Adjust this condition for more complex buffering
                    buffered_data[i, j] = buffer_distance  # Assign the buffer distance as the pixel value
        
        # Write the buffered raster to an output file
        profile.update(dtype=rasterio.float32, count=1)
        with rasterio.open(output_raster, 'w', **profile) as dst:
            dst.write(buffered_data, 1)  # Write the first band of the buffered raster

# Example usage





In [23]:

buffer_list = (10,20,30,40,50,60,70,80,90,100,200)

base_folder = 'N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/test/hazard_map'
distribution_raster = "N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f05_distribution/f02_distribution_model_luxembourg/heracleum_mantegazzianum_RandomForest_model_v2.tif"

for buf in buffer_list:
    
    input_raster = distribution_raster  
    buffer_distance = buf  
    output_raster = base_folder +"/"+ 'single'+"/"+'tmp_buffer_'+str(buffer_distance)+"_heracl.tif"
    buffer_raster(input_raster, output_raster, buffer_distance)




In [24]:
# read and set up raster list
import glob
import os

# Define the folder path
folder_path = r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\test\hazard_map\single"

# Create a list of all .tif files in the folder
raster_list = glob.glob(os.path.join(folder_path, "*.tif"))

# Print the list of rasters
print("Raster List:")
for raster in raster_list:
    print(raster)

Raster List:
N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\test\hazard_map\single\tmp_buffer_100_heracl.tif
N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\test\hazard_map\single\tmp_buffer_10_heracl.tif
N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\test\hazard_map\single\tmp_buffer_200_heracl.tif
N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\test\hazard_map\single\tmp_buffer_20_heracl.tif
N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\test\hazard_map\single\tmp_buffer_30_heracl.tif
N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\test\hazard_map\single\tmp_buffer_40_heracl.tif
N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\test\hazard_map\single\tmp_buffer_50_heracl.tif
N:\C2205_FAIRiCUBE\f02_data\d060_data_LU

In [25]:
import rasterio
import numpy as np

def overlay_rasters(raster_list, output_raster):
    """
    Overlay multiple buffer rasters and generate a final raster where each pixel contains the lowest buffer distance.
    
    Args:
        raster_list (list): List of file paths to input raster files.
        output_raster (str): File path to save the resulting raster.
    """
    # Open the first raster to use as a template
    with rasterio.open(raster_list[0]) as src:
        profile = src.profile
        profile.update(dtype=rasterio.float32)  # Ensure the output is float32
        # Read the first raster data
        combined_data = src.read(1).astype(np.float32)
    
    # Initialize the combined raster with high values (to find the minimum distance)
    combined_data[combined_data > 0] = np.inf  # Set initial non-zero buffer values to infinity
    
    # Iterate over each raster in the list
    for raster_path in raster_list:
        with rasterio.open(raster_path) as src:
            # Read current raster data
            data = src.read(1).astype(np.float32)
            # Combine using minimum values (ignoring zero background)
            combined_data = np.minimum(combined_data, np.where(data > 0, data, np.inf))
    
    # Replace infinity with zero for non-buffered areas
    combined_data[np.isinf(combined_data)] = 0
    
    # Write the resulting raster to disk
    with rasterio.open(output_raster, 'w', **profile) as dst:
        dst.write(combined_data, 1)

# Example Usage
raster_list = raster_list

output_raster = base_folder +"/"+"final_buffer_distance.tif"

overlay_rasters(raster_list, output_raster)





## overlay buffer raster with poi raster

In [26]:
import rasterio
import numpy as np

# Define file paths
buffer_raster_path = output_raster
hospital_raster_path = r'N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f02_raster_10m_collection\aoi_osm_2024_10m_b4.tif'
output_raster_path = base_folder +"/"+"final_buffer_distance_pois.tif"

# Open buffer raster
with rasterio.open(buffer_raster_path) as buffer_src:
    buffer_data = buffer_src.read(1)  # Read the first band
    buffer_meta = buffer_src.meta  # Get metadata

# Open hospital raster
with rasterio.open(hospital_raster_path) as hospital_src:
    hospital_data = hospital_src.read(1)  # Read the first band

# Ensure the rasters have the same shape
if buffer_data.shape != hospital_data.shape:
    raise ValueError("Rasters must have the same dimensions for overlaying.")

# Overlay operation: if hospital pixel is 1, use 1, otherwise use buffer raster value
final_raster_data = np.where(hospital_data == 1, 1, buffer_data)

# Update metadata for the output raster
buffer_meta.update(dtype=rasterio.float32, compress="lzw")

# Write the result to a new file
with rasterio.open(output_raster_path, "w", **buffer_meta) as dest:
    dest.write(final_raster_data, 1)

print(f"Overlay raster saved to {output_raster_path}")


ValueError: Rasters must have the same dimensions for overlaying.